## Deliverable 2. Create a Customer Travel Destinations Map

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

In [5]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,1,Tuatapere,NZ,-46.1333,167.6833,54.12,60,39,8.97,scattered clouds
1,3,Butaritari,KI,3.0707,172.7902,82.71,71,84,15.77,broken clouds
2,5,Ugento,IT,39.9263,18.1574,56.70,65,100,26.66,overcast clouds
3,7,Bredasdorp,ZA,-34.5322,20.0403,50.29,90,100,4.41,overcast clouds
4,9,Podporozhye,RU,60.9112,34.1706,34.63,94,9,2.80,clear sky


In [15]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter minimum temperature."))
max_temp = float(input("Enter maximum temperature."))


Enter minimum temperature.75
Enter maximum temperature.100


In [16]:
print (f"You entered min temp as {min_temp} and max temp as {max_temp}")

You entered min temp as 75.0 and max temp as 100.0


In [17]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_city_data_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & 
                            (city_data_df['Max Temp'] <= max_temp)]
filtered_city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,3,Butaritari,KI,3.0707,172.7902,82.71,71,84,15.77,broken clouds
8,17,Hilo,US,19.7297,-155.0900,78.80,82,75,0.00,light rain
13,27,Lorengau,PG,-2.0226,147.2712,86.34,68,24,8.86,few clouds
14,29,Georgetown,MY,5.4112,100.3354,84.13,88,20,1.99,few clouds
17,35,Kavieng,PG,-2.5744,150.7967,84.51,73,37,9.91,light rain
...,...,...,...,...,...,...,...,...,...,...
670,1337,Amahai,ID,-3.3333,128.9167,85.32,67,81,3.00,broken clouds
672,1341,Auki,SB,-8.7676,160.7034,83.75,75,100,1.61,overcast clouds
674,1345,Kudahuvadhoo,MV,2.6708,72.8944,82.90,76,100,23.94,overcast clouds
675,1347,Pochutla,MX,15.7432,-96.4661,78.53,87,53,4.50,broken clouds


In [18]:
# Determine if there are any empty rows.
filtered_city_data_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [19]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_city_data_df = filtered_city_data_df.dropna()
filtered_city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,3,Butaritari,KI,3.0707,172.7902,82.71,71,84,15.77,broken clouds
8,17,Hilo,US,19.7297,-155.0900,78.80,82,75,0.00,light rain
13,27,Lorengau,PG,-2.0226,147.2712,86.34,68,24,8.86,few clouds
14,29,Georgetown,MY,5.4112,100.3354,84.13,88,20,1.99,few clouds
17,35,Kavieng,PG,-2.5744,150.7967,84.51,73,37,9.91,light rain


In [20]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
650,Caravelas,BR,76.23,broken clouds,-17.7125,-39.2481,
651,Gizo,SB,85.60,broken clouds,-8.1030,156.8419,
659,Kaura Namoda,NG,79.77,few clouds,12.5898,6.5779,
660,Sakakah,SA,78.37,broken clouds,29.9697,40.2064,
668,Sabya,SA,82.02,clear sky,17.1495,42.6254,
670,Amahai,ID,85.32,broken clouds,-3.3333,128.9167,
672,Auki,SB,83.75,overcast clouds,-8.7676,160.7034,
674,Kudahuvadhoo,MV,82.90,overcast clouds,2.6708,72.8944,
675,Pochutla,MX,78.53,broken clouds,15.7432,-96.4661,
677,Goure,NE,87.03,overcast clouds,13.9835,10.2704,


In [21]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    #Add to params
    params["location"] = f"{lat},{lng}"
        
    #6d: Send gmaps request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params = params).json()
        
    #Get first result if available
    try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
            print(str(index) + " " + hotels["results"][0]["name"])
    except (IndexError): 
            print("Hotel not found ... skipping.")
    

print ('\n-----------------------------')
print ('Got the hotels for you!')
print ('-----------------------------')

1 Isles Sunset Lodge
8 Hilo Hawaiian Hotel
13 Lorengau Harbourside Hotel
14 Cititel Penang
17 Nusa Island Retreat
22 Rydges Port Macquarie
24 The Hamsa Resort
38 Hôtel Le Dronmi
39 Villa Enata
42 Hotel California Todos Santos
49 Hotel Xaguate
50 Bora-Bora Pearl Beach Resort & Spa
59 Dibbo areaمنطقة دبو
66 The Irish Village
67 Kamaneahn Hotel Inc
84 Sheraton Kauai Coconut Beach Resort
86 Screensaver
90 The Esplanade Hotel
95 Skye-Jace Transient
97 Kashmiri Hotal
98 The Espiritu Hotel
99 Central Hotel
101 Rio Xingu Camping
106 Hotel Citra
107 Keleti Beach Resort
110 Hotel Muesse
116 Grage Hotel Bengkulu
118 OYO 991 Duy Anh Hotel
120 Kost Pria
124 HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel
127 Hotel Kununurra
133 Taba Hotel
137 Adinkra Lodge
140 Hostel Aloha Brasil
141 RUMBIA RESORT VILLA
142 Hotel Albergue de Olinda
144 Radisson Grenada Beach Resort
154 Oota Ryokan
157 La Quinta Inn by Wyndham Abilene
164 The Regency Hotel
166 Leonardo Family Beach House 1
168 Paradise Inn
170 Fengshui Res

In [22]:
# Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [23]:
# Create the output File (CSV)
output_path = "Vacation_Search/WeatherPy_vacation.csv"

# Export the City_Data into a csv
hotel_df.to_csv(output_path, index_label="City_ID")

In [24]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center = (30, 31), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp,
                                dissipating = False, 
                                max_intensity = 300,
                                point_radius = 4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

#  Display the figure
fig

Figure(layout=FigureLayout(height='420px'))